In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
972,What a good movie! At last a picture revealing...,positive
305,The movie is about two brothers that are suppo...,negative
889,"""Journey of Hope"" tells of a poor Turkish fami...",positive
665,"Unfortunately, one of the best efforts yet mad...",negative
267,An excellent documentry. I personally remember...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\91704\AppData\Local\Temp\ipykernel_876\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
972,good movie last picture revealing unknown side...,positive
305,movie two brother supposed alike way expect sm...,negative
889,journey hope tell poor turkish family odyssey ...,positive
665,unfortunately one best effort yet made area sp...,negative
267,excellent documentry personally remember growi...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
positive    251
negative    249
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
972,good movie last picture revealing unknown side...,1
305,movie two brother supposed alike way expect sm...,0
889,journey hope tell poor turkish family odyssey ...,1
665,unfortunately one best effort yet made area sp...,0
267,excellent documentry personally remember growi...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/maskedwolf4/YT_Comments_Sentiment_Analysis.mlflow')
dagshub.init(repo_owner='maskedwolf4', repo_name='YT_Comments_Sentiment_Analysis', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e34a94e0-c140-4640-9758-90f6ae8043c4&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=48f576f816ded9c754592e5a8816ab610b15f1b1d14bacc1d377a28c236b9401




Accessing as maskedwolf4

Initialized MLflow to track repo "maskedwolf4/YT_Comments_Sentiment_Analysis"

Repository maskedwolf4/YT_Comments_Sentiment_Analysis initialized!

2025/05/14 07:33:51 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/35db92a849b949dcb179ac69a9f03dbe', creation_time=1747188284926, experiment_id='0', last_update_time=1747188284926, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-14 07:34:04,842 - INFO - Starting MLflow run...
2025-05-14 07:34:07,000 - INFO - Logging preprocessing parameters...
2025-05-14 07:34:08,319 - INFO - Initializing Logistic Regression model...
2025-05-14 07:34:08,324 - INFO - Fitting the model...
2025-05-14 07:34:08,645 - INFO - Model training complete.
2025-05-14 07:34:08,647 - INFO - Logging model parameters...
2025-05-14 07:34:09,127 - INFO - Making predictions...
2025-05-14 07:34:09,133 - INFO - Calculating evaluation metrics...
2025-05-14 07:34:09,177 - INFO - Logging evaluation metrics...
2025-05-14 07:34:10,877 - INFO - Saving and logging the model...
2025/05/14 07:34:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-14 07:35:03,529 - INFO - Model training and logging completed in 56.53 seconds.
2025-05-14 07:35:03,534 - INFO - Accuracy: 0.592
2025-05-14 07:35:03,540 - INFO - Precision: 

🏃 View run bouncy-penguin-960 at: https://dagshub.com/maskedwolf4/YT_Comments_Sentiment_Analysis.mlflow/#/experiments/0/runs/c66915f7e06f4f33ae85837e760bcdbd
🧪 View experiment at: https://dagshub.com/maskedwolf4/YT_Comments_Sentiment_Analysis.mlflow/#/experiments/0
